In [19]:
import pandas as pd
import matplotlib as plt

In [37]:
# detect coding of csv file

import chardet

with open("AviationData.csv", "rb") as f:
    result = chardet.detect(f.read(100000))  # Read a sample of the file
    print(result)  # Example output: {'encoding': 'Windows-1252', 'confidence': 0.99}

#df = pd.read_csv("AviationData.csv", encoding=result['encoding'])


{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [41]:
# due to source of data columns 6,7 and 28 have mixed dtypes so low_memory 
# flag is set to False. fortunately this data size is not to large for my memory size. 
#for better control, chunksize flag can be specified
aviation_data = pd.read_csv('AviationData.csv', encoding = 'cp1252', low_memory=False)
aviation_data.head(5)

Event.Id Investigation.Type Accident.Number  Event.Date  \
0  20001218X45444           Accident      SEA87LA080  1948-10-24   
1  20001218X45447           Accident      LAX94LA336  1962-07-19   
2  20061025X01555           Accident      NYC07LA005  1974-08-30   
3  20001218X45448           Accident      LAX96LA321  1977-06-19   
4  20041105X01764           Accident      CHI79FA064  1979-08-02   

          Location        Country   Latitude   Longitude Airport.Code  \
0  MOOSE CREEK, ID  United States        NaN         NaN          NaN   
1   BRIDGEPORT, CA  United States        NaN         NaN          NaN   
2    Saltville, VA  United States  36.922223  -81.878056          NaN   
3       EUREKA, CA  United States        NaN         NaN          NaN   
4       Canton, OH  United States        NaN         NaN          NaN   

  Airport.Name  ... Purpose.of.flight Air.carrier Total.Fatal.Injuries  \
0          NaN  ...          Personal         NaN                  2.0   
1          NaN  ...          Personal         NaN                  4.0   
2          NaN  ...          Personal         NaN                  3.0   
3          NaN  ...          Personal         NaN                  2.0   
4          NaN  ...          Personal         NaN                  1.0   

  Total.Serious.Injuries Total.Minor.Injuries Total.Uninjured  \
0                    0.0                  0.0             0.0   
1                    0.0                  0.0             0.0   
2                    NaN                  NaN             NaN   
3                    0.0                  0.0             0.0   
4                    2.0                  NaN             0.0   

  Weather.Condition  Broad.phase.of.flight   Report.Status Publication.Date  
0               UNK                 Cruise  Probable Cause              NaN  
1               UNK                Unknown  Probable Cause       19-09-1996  
2               IMC                 Cruise  Probable Cause       26-02-2007  
3               IMC                 Cruise  Probable Cause       12-09-2000  
4               VMC               Approach  Probable Cause       16-04-1980  

[5 rows x 31 columns]

In [47]:
# creating subset of aviation_data that are relevant to analyzing safety risk. may add more if necassary

data_subset = aviation_data.iloc[:,[0,2,3,4,10,11,12,14,15,17,18,21,23,24,25,26,27,30]]
data_subset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88889 entries, 0 to 88888
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Event.Id                88889 non-null  object 
 1   Accident.Number         88889 non-null  object 
 2   Event.Date              88889 non-null  object 
 3   Location                88837 non-null  object 
 4   Injury.Severity         87889 non-null  object 
 5   Aircraft.damage         85695 non-null  object 
 6   Aircraft.Category       32287 non-null  object 
 7   Make                    88826 non-null  object 
 8   Model                   88797 non-null  object 
 9   Number.of.Engines       82805 non-null  float64
 10  Engine.Type             81812 non-null  object 
 11  Purpose.of.flight       82697 non-null  object 
 12  Total.Fatal.Injuries    77488 non-null  float64
 13  Total.Serious.Injuries  76379 non-null  float64
 14  Total.Minor.Injuries    76956 non-null

In [54]:
data_subset.head(10)

Event.Id Accident.Number  Event.Date          Location  \
0  20001218X45444      SEA87LA080  1948-10-24   MOOSE CREEK, ID   
1  20001218X45447      LAX94LA336  1962-07-19    BRIDGEPORT, CA   
2  20061025X01555      NYC07LA005  1974-08-30     Saltville, VA   
3  20001218X45448      LAX96LA321  1977-06-19        EUREKA, CA   
4  20041105X01764      CHI79FA064  1979-08-02        Canton, OH   
5  20170710X52551      NYC79AA106  1979-09-17        BOSTON, MA   
6  20001218X45446      CHI81LA106  1981-08-01        COTTON, MN   
7  20020909X01562      SEA82DA022  1982-01-01       PULLMAN, WA   
8  20020909X01561      NYC82DA015  1982-01-01  EAST HANOVER, NJ   
9  20020909X01560      MIA82DA029  1982-01-01  JACKSONVILLE, FL   

  Injury.Severity Aircraft.damage Aircraft.Category               Make  \
0        Fatal(2)       Destroyed               NaN            Stinson   
1        Fatal(4)       Destroyed               NaN              Piper   
2        Fatal(3)       Destroyed               NaN             Cessna   
3        Fatal(2)       Destroyed               NaN           Rockwell   
4        Fatal(1)       Destroyed               NaN             Cessna   
5       Non-Fatal     Substantial          Airplane  Mcdonnell Douglas   
6        Fatal(4)       Destroyed               NaN             Cessna   
7       Non-Fatal     Substantial          Airplane             Cessna   
8       Non-Fatal     Substantial          Airplane             Cessna   
9       Non-Fatal     Substantial               NaN     North American   

          Model  Number.of.Engines    Engine.Type Purpose.of.flight  \
0         108-3                1.0  Reciprocating          Personal   
1      PA24-180                1.0  Reciprocating          Personal   
2          172M                1.0  Reciprocating          Personal   
3           112                1.0  Reciprocating          Personal   
4           501                NaN            NaN          Personal   
5           DC9                2.0      Turbo Fan               NaN   
6           180                1.0  Reciprocating          Personal   
7           140                1.0  Reciprocating          Personal   
8          401B                2.0  Reciprocating          Business   
9  NAVION L-17B                1.0  Reciprocating          Personal   

   Total.Fatal.Injuries  Total.Serious.Injuries  Total.Minor.Injuries  \
0                   2.0                     0.0                   0.0   
1                   4.0                     0.0                   0.0   
2                   3.0                     NaN                   NaN   
3                   2.0                     0.0                   0.0   
4                   1.0                     2.0                   NaN   
5                   NaN                     NaN                   1.0   
6                   4.0                     0.0                   0.0   
7                   0.0                     0.0                   0.0   
8                   0.0                     0.0                   0.0   
9                   0.0                     0.0                   3.0   

   Total.Uninjured Weather.Condition Publication.Date  
0              0.0               UNK              NaN  
1              0.0               UNK       19-09-1996  
2              NaN               IMC       26-02-2007  
3              0.0               IMC       12-09-2000  
4              0.0               VMC       16-04-1980  
5             44.0               VMC       19-09-2017  
6              0.0               IMC       06-11-2001  
7              2.0               VMC       01-01-1982  
8              2.0               IMC       01-01-1982  
9              0.0               IMC       01-01-1982

In [51]:
data_subset['Aircraft.damage'].unique()

array(['Destroyed', 'Substantial', 'Minor', nan, 'Unknown'], dtype=object)